#Lab1
##0-1Knapsack

In [104]:
import numpy as np
from icecream import ic
from tqdm.notebook import tqdm
ic.configureOutput(includeContext=False, outputFunction=tqdm.write)

In [105]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [106]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [107]:
print(VALUES)
print(WEIGHTS)
CONSTRAINTS

[37 80 50 99 74 19  1 25 92 78]
[[43  7]
 [14 50]
 [59 72]
 [50 87]
 [64 39]
 [30 36]
 [30  5]
 [42 58]
 [70 65]
 [56 87]]


array([[108,  89],
       [198, 161],
       [299, 310]])

###TEST PROBLEMS

In [ ]:
#test some solutions
#start from classic knapsack solution
#random solution
def random_feasible_init(values, weights, constraints, num_knapsacks, rng=None):
    rng = np.random.default_rng() if rng is None else rng
    N, D = weights.shape; K = num_knapsacks
    sol = np.zeros((N, K), dtype=int)
    loads = np.zeros((K, D), dtype=int)

    # from low to high value/weight ratio
    norm = np.maximum(constraints.astype(float), 1.0)
    score = values / ((weights / norm).sum(axis=1) + 1e-9)
    order = np.argsort(score)                   

    for i in order:
        k = rng.integers(K)                     
        if np.all(loads[k] + weights[i] <= constraints):
            sol[i, k] = 1
            loads[k] += weights[i]
    return sol



In [109]:
def fitness(sol, values):
    # only return the fitness value
    return int((sol * values[:, None]).sum())

def knapsack_loads(sol, weights):
    return sol.T @ weights  # (K, D)

In [ ]:
#local search
#neighbor
def best_improving_neighbor(sol, values, weights, constraints):
    import numpy as np
    N, K = sol.shape
    cur_fit = fitness(sol, values)
    loads = knapsack_loads(sol, weights)

    # 1. add i to some bag k
    for i in range(N):
        if sol[i].any():
            continue
        wi = weights[i]
        for k in range(K):
            if np.all(loads[k] + wi <= constraints):
                new_sol = sol.copy()
                new_sol[i, :] = 0
                new_sol[i, k] = 1
                gain = int(fitness(new_sol, values) - cur_fit)
                if gain > 0:
                    return new_sol, gain, {"op": "add", "i": i, "j": None, "k_from": None, "k_to": k}

    # 2. swap： swap i to bag k's j
    for k in range(K):
        for j in range(N):
            if sol[j, k] != 1:
                continue
            for i in range(N):
                if sol[i].any():
                    continue
                if np.all(loads[k] - weights[j] + weights[i] <= constraints):
                    new_sol = sol.copy()
                    new_sol[j, :] = 0
                    new_sol[i, :] = 0
                    new_sol[i, k] = 1
                    gain = int(fitness(new_sol, values) - cur_fit)
                    if gain > 0:
                        return new_sol, gain, {"op": "swap", "i": i, "j": j, "k_from": k, "k_to": k}

    return None, 0, None


In [ ]:
from tqdm.auto import tqdm
from icecream import ic
ic.configureOutput(includeContext=False, outputFunction=tqdm.write)

def hill_climb(values, weights, constraints, num_knapsacks, *,
               restarts=20, max_iters=500, seed=42,
               debug=False, log_every=500,
               refresh_every=200):   
    rng = np.random.default_rng(seed)
    best_sol, best_fit = None, -1

    # total iterations for the progress bar
    TOTAL = restarts * max_iters
    bar = tqdm(total=TOTAL, desc="search", unit="try",
               dynamic_ncols=True, leave=True, position=0)
    bar.refresh()  # enforce refresh

    tries_done = 0  # total tries done so far

    for r in range(restarts):
        sol = random_feasible_init(values, weights, constraints, num_knapsacks, rng)
        cur_fit = fitness(sol, values)

        it = 0
        while it < max_iters:
            tries_done += 1
            bar.update(1)

            if tries_done % refresh_every == 0:
                bar.set_postfix_str(f"r={r+1} fit={int(cur_fit)}")
                bar.refresh()  

            neighbor, gain, info = best_improving_neighbor(sol, values, weights, constraints)
            if neighbor is None or gain <= 0:
                break  

            # accept the neighbor
            sol = neighbor
            cur_fit = fitness(sol, values)
            it += 1

            if debug and (it % log_every == 0):
                ic(f"r{r+1} it={it} {info.get('op')} i={info.get('i')} "
                   f"j={info.get('j')} k={info.get('k_to')} "
                   f"gain=+{int(gain)} fit={int(cur_fit)}")

        if cur_fit > best_fit:
            best_fit, best_sol = cur_fit, sol

    #end bar
    bar.total = bar.n
    bar.close()
    return best_sol, int(best_fit)


In [ ]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)



sol = random_feasible_init(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, rng)
print("Initial solution:")
print(sol)
sol, fit = hill_climb(
    VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
    restarts=20, max_iters=5000, seed=42,
    debug=False    
)






Initial solution:
[[0 1 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 1 0]]


search: 100%|██████████| 34/34 [00:00<00:00, 4853.03try/s]


In [ ]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)




sol = random_feasible_init(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, rng)
print("Initial solution:")
print(sol)
sol, fit = hill_climb(
    VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
    restarts=20, max_iters=5000, seed=42,
    debug=False     
)



Initial solution:
[[0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 

search: 100%|██████████| 1905/1905 [00:06<00:00, 272.66try/s, r=19 fit=35417]  


In [ ]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)



sol = random_feasible_init(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, rng)
print("Initial solution:")
print(sol)
sol, fit = hill_climb(
    VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
    restarts=20, max_iters=50, seed=42,
    debug=True 
)



Initial solution:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


search: 100%|██████████| 1000/1000 [01:04<00:00, 15.59try/s, r=20 fit=606978]


In [115]:
print(VALUES)
print(WEIGHTS)
print(CONSTRAINTS)

[ 89 773 654 ... 819 678 498]
[[197  33 627 ... 319 937 293]
 [550 543 242 ... 942 568 950]
 [436 411 836 ... 664 169 846]
 ...
 [869 582 968 ... 567 291 288]
 [ 97 220 662 ... 171 310 833]
 [656  51 146 ... 286 725 372]]
[79834 52005 17527 30007 59295 98921 12881 36174 63958 90566 25740 12251
 27746 78136 90704 39249 23072 69010 17326 29277 42886 48016 74931 93479
 30898 18137 75557 56030 75506 90246 42093 34668 22075 79709 98971 15032
 57157 21369 76865 11589 82666 78074 68715 85558 61242 80943 26905 84378
 85069 77221 16434 28156 17827 83548 72921 27512 41395 23936 72519 96465
 12904 72709 81023 81015 28959 98041 90482 89802 76408 29899 51171 94767
 78351 57860 58504 40545 97025 99185 42888 54965 24945 51281 31276 61913
 96714 93034 72901 95222 37857 53461 15463 71355 64979 70086 48286 38026
 60109 78500 56643 69915]


In [116]:
#sample data

rng = np.random.default_rng(0)
NUM_KNAPSACKS = 2
NUM_ITEMS = 3
NUM_DIMENSIONS = 2

VALUES   = rng.integers(0, 10, size=NUM_ITEMS)              # (3,)
WEIGHTS  = rng.integers(0, 10, size=(NUM_ITEMS, NUM_DIMENSIONS))  # (3,2)
# 单背包/共享容量（不推荐多背包用）
CONSTRAINTS_shared = rng.integers(5, 15, size=NUM_DIMENSIONS)      # (2,)
# 多背包容量（推荐多背包用）
CONSTRAINTS_multi  = rng.integers(5, 15, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))  # (2,2)

print("VALUES.shape:", VALUES.shape)            # (3,)
print("WEIGHTS.shape:", WEIGHTS.shape)          # (3,2)
print("共享容量:", CONSTRAINTS_shared.shape)     # (2,)
print("多背包容量:", CONSTRAINTS_multi.shape)    # (2,2)

print("VALUES =", VALUES)
print("WEIGHTS=\n", WEIGHTS)
print("CONSTRAINTS_multi=\n", CONSTRAINTS_multi)


VALUES.shape: (3,)
WEIGHTS.shape: (3, 2)
共享容量: (2,)
多背包容量: (2, 2)
VALUES = [8 6 5]
WEIGHTS=
 [[2 3]
 [0 0]
 [0 1]]
CONSTRAINTS_multi=
 [[14 10]
 [11 14]]
